# Infilling COVID recovery data
First, we must clean the data. Several columns are not GHGs so we remove them. 

In [23]:
import pandas as pd
import pyam
import silicone.utils
import silicone.database_crunchers as dc

In [2]:
input = pd.read_excel("../input/CompiledPiersScenarios_distri_with_cleaning.xlsx", header=[0, 1])
sr15_data = silicone.utils.download_or_load_sr15("../input/complete_sr15.csv")
convert_n2o = (2 * 14 + 16) / (2 * 14) 
convert_co2 = (12 + 2 * 16) / 12
input

pyam.utils - INFO: Reading `../input/complete_sr15.csv`


,model,scenario,year,co2_fossil,co2_land,ch4,n2o,sox,co,nmvoc,...,mcf,hcfc22,hcfc141b,hcfc142b,halon1211,halon1202,halon1301,halon2402,ch3br,ch3cl
,Unnamed: 0_level_1,Unnamed: 1_level_1,Unnamed: 2_level_1,GtC/yr,GtC/yr,Mt/yr,Mt/yr [units of N],Mt/yr [units of S],Mt/yr,Mt/yr,...,kt/yr,kt/yr,kt/yr,kt/yr,kt/yr,kt/yr,kt/yr,kt/yr,kt/yr,kt/yr
0,emissions_ndc_month_base,Baseline,2020,9.401228,1.384467,324.937346,6.081999,35.540802,756.389584,167.700559,...,0,444.821,57.463,18.867,2.531,0,0.911,0.055,155.925,3511.082
1,emissions_ndc_month_base,Baseline,2030,10.406026,0.858738,338.217710,6.750341,34.062300,721.864751,161.375566,...,0,472.764,58.080,9.424,1.225,0,0.545,0.017,155.925,3511.082
2,emissions_ndc_month_base,Baseline,2040,11.224362,0.211322,316.024645,6.862659,28.229820,652.913333,149.261614,...,0,240.141,44.431,4.563,0.593,0,0.326,0.005,155.925,3511.082
3,emissions_ndc_month_g1_3,Green-recovery,2020,9.401228,1.384467,324.937346,6.081999,35.540802,756.389584,167.700559,...,0,444.821,57.463,18.867,2.531,0,0.911,0.055,155.925,3511.082
4,emissions_ndc_month_g1_3,Green-recovery,2030,6.671190,0.553782,217.080547,4.330161,21.865040,463.442623,103.611824,...,0,472.764,58.080,9.424,1.225,0,0.545,0.017,155.925,3511.082
5,emissions_ndc_month_g2_2,Fossil-recovery,2020,9.401228,1.384467,324.937346,6.081999,35.540802,756.389584,167.700559,...,0,444.821,57.463,18.867,2.531,0,0.911,0.055,155.925,3511.082
6,emissions_ndc_month_g2_2,Fossil-recovery,2030,11.446629,0.944612,372.039481,7.425375,37.468530,794.051226,177.513122,...,0,472.764,58.080,9.424,1.225,0,0.545,0.017,155.925,3511.082
7,emissions_ndc_month_g2_2,Fossil-recovery,2040,12.346798,0.232455,347.627110,7.548925,31.052802,718.204666,164.187775,...,0,240.141,44.431,4.563,0.593,0,0.326,0.005,155.925,3511.082


We need to reshape this into the correct structure for pyam

In [3]:
input = input.set_index(keys=[input.columns[0], input.columns[1], input.columns[2]])

In [4]:
input = input.stack(level=[0, 1]).reset_index()
input

,"(model, Unnamed: 0_level_1)","(scenario, Unnamed: 1_level_1)","(year, Unnamed: 2_level_1)",level_3,level_4,0
0,emissions_ndc_month_base,Baseline,2020,bc,Mt/yr,5.768693
1,emissions_ndc_month_base,Baseline,2020,c2f6,kt/yr,1.557000
2,emissions_ndc_month_base,Baseline,2020,c6f14,kt/yr,0.347200
3,emissions_ndc_month_base,Baseline,2020,carb_tet,kt/yr,0.000000
4,emissions_ndc_month_base,Baseline,2020,cf4,kt/yr,10.949979
...,...,...,...,...,...,...
307,emissions_ndc_month_g2_2,Fossil-recovery,2040,nmvoc,Mt/yr,164.187775
308,emissions_ndc_month_g2_2,Fossil-recovery,2040,nox,Mt/yr [units of N],36.895177
309,emissions_ndc_month_g2_2,Fossil-recovery,2040,oc,Mt/yr,25.661602
310,emissions_ndc_month_g2_2,Fossil-recovery,2040,sf6,kt/yr,2.960982


In [5]:
input = input.rename(
    {
        input.columns[0]: "model", 
        input.columns[1]: "scenario", 
        input.columns[2]: "year", 
        input.columns[3]: "variable",
        input.columns[4]: "unit", 
        input.columns[5]: "value"
    }, 
    axis= 1)
input["region"] = "World"

In [6]:
input_df = pyam.IamDataFrame(input)
unwanted_cols = ["sox", "co", "nmvoc", "nox", "bc", "oc", "cfc113", "carb_tet", "cfc114", "mcf", "halon1202"]
input_df.filter(variable=unwanted_cols, keep=False, inplace=True)
input_df.data.head(30)

,model,scenario,region,variable,unit,year,value
3,emissions_ndc_month_base,Baseline,World,c2f6,kt/yr,2020,1.557000
4,emissions_ndc_month_base,Baseline,World,c2f6,kt/yr,2030,1.549600
5,emissions_ndc_month_base,Baseline,World,c2f6,kt/yr,2040,1.465300
6,emissions_ndc_month_base,Baseline,World,c6f14,kt/yr,2020,0.347200
7,emissions_ndc_month_base,Baseline,World,c6f14,kt/yr,2030,0.345400
8,emissions_ndc_month_base,Baseline,World,c6f14,kt/yr,2040,0.326700
12,emissions_ndc_month_base,Baseline,World,cf4,kt/yr,2020,10.949979
13,emissions_ndc_month_base,Baseline,World,cf4,kt/yr,2030,9.483124
14,emissions_ndc_month_base,Baseline,World,cf4,kt/yr,2040,10.591102
15,emissions_ndc_month_base,Baseline,World,cfc11,kt/yr,2020,30.725000


Convert units. We need to change the format of the unit names first so they are recognised by pyam. 

In [7]:
input_df.convert_unit("Mt/yr [units of N]", "Mt/yr", convert_n2o, inplace=True)
input_df.convert_unit("GtC/yr", "GtCO2/yr", convert_co2, inplace=True)

In [8]:
orig_units = input_df.variables(True)
unit_rename_dict = {
    orig_units["variable"].iloc[i]: orig_units["unit"].iloc[i][:2] + " " + 
    orig_units["variable"].iloc[i].upper().replace("_FOSSIL", "").replace("_LAND", "").replace("HFC43_10", "HFC4310mee") + "/yr" for i in range(len(orig_units))
}

In [9]:
for i in input_df.data.index:
    input_df.data.loc[i, "unit"] = unit_rename_dict[input_df.data.loc[i, "variable"]]

In [10]:
input_df.data

,model,scenario,region,variable,unit,year,value
3,emissions_ndc_month_base,Baseline,World,c2f6,kt C2F6/yr,2020,1.557000
4,emissions_ndc_month_base,Baseline,World,c2f6,kt C2F6/yr,2030,1.549600
5,emissions_ndc_month_base,Baseline,World,c2f6,kt C2F6/yr,2040,1.465300
6,emissions_ndc_month_base,Baseline,World,c6f14,kt C6F14/yr,2020,0.347200
7,emissions_ndc_month_base,Baseline,World,c6f14,kt C6F14/yr,2030,0.345400
...,...,...,...,...,...,...,...
295,emissions_ndc_month_g2_2,Fossil-recovery,World,nh3,Mt NH3/yr,2030,51.139301
296,emissions_ndc_month_g2_2,Fossil-recovery,World,nh3,Mt NH3/yr,2040,55.485087
306,emissions_ndc_month_g2_2,Fossil-recovery,World,sf6,kt SF6/yr,2020,6.318104
307,emissions_ndc_month_g2_2,Fossil-recovery,World,sf6,kt SF6/yr,2030,6.659998


In [11]:
converted_input = silicone.utils.convert_units_to_MtCO2_equiv(input_df, use_ar4_data=False)

pyam.logging - WARNING: Using a dictionary to convert units is deprecated and will be removed in future versions. Use explicit keyword arguments instead!
pyam.logging - WARNING: Using a dictionary to convert units is deprecated and will be removed in future versions. Use explicit keyword arguments instead!
pyam.logging - WARNING: Using a dictionary to convert units is deprecated and will be removed in future versions. Use explicit keyword arguments instead!
pyam.logging - WARNING: Using a dictionary to convert units is deprecated and will be removed in future versions. Use explicit keyword arguments instead!
pyam.logging - WARNING: Using a dictionary to convert units is deprecated and will be removed in future versions. Use explicit keyword arguments instead!
pyam.logging - WARNING: Using a dictionary to convert units is deprecated and will be removed in future versions. Use explicit keyword arguments instead!
pyam.logging - WARNING: Using a dictionary to convert units is deprecated an

In [12]:
converted_input.data.head(10)

,model,scenario,region,variable,unit,year,value
3,emissions_ndc_month_base,Baseline,World,c2f6,Mt CO2-equiv/yr,2020,17.282700
4,emissions_ndc_month_base,Baseline,World,c2f6,Mt CO2-equiv/yr,2030,17.200560
5,emissions_ndc_month_base,Baseline,World,c2f6,Mt CO2-equiv/yr,2040,16.264830
6,emissions_ndc_month_base,Baseline,World,c6f14,Mt CO2-equiv/yr,2020,2.746352
7,emissions_ndc_month_base,Baseline,World,c6f14,Mt CO2-equiv/yr,2030,2.732114
8,emissions_ndc_month_base,Baseline,World,c6f14,Mt CO2-equiv/yr,2040,2.584197
12,emissions_ndc_month_base,Baseline,World,cf4,Mt CO2-equiv/yr,2020,72.598361
13,emissions_ndc_month_base,Baseline,World,cf4,Mt CO2-equiv/yr,2030,62.873112
14,emissions_ndc_month_base,Baseline,World,cf4,Mt CO2-equiv/yr,2040,70.219009
15,emissions_ndc_month_base,Baseline,World,cfc11,Mt CO2-equiv/yr,2020,143.178500


# Working with the consistent units
We now need to construct a Kyoto total for this and can then split this into the relevant components. 

In [13]:
kyoto_input = silicone.utils._construct_consistent_values(
    "Kyoto Gases (AR5-GWP100)", converted_input.variables().to_list(), converted_input
)

In [14]:
kyoto_input.data.head()

,model,scenario,region,variable,unit,year,value
0,emissions_ndc_month_base,Baseline,World,Kyoto Gases (AR5-GWP100),Mt CO2-equiv/yr,2020,83562.337349
1,emissions_ndc_month_base,Baseline,World,Kyoto Gases (AR5-GWP100),Mt CO2-equiv/yr,2030,88260.272949
2,emissions_ndc_month_base,Baseline,World,Kyoto Gases (AR5-GWP100),Mt CO2-equiv/yr,2040,90328.887510
3,emissions_ndc_month_g1_3,Green-recovery,World,Kyoto Gases (AR5-GWP100),Mt CO2-equiv/yr,2020,83562.337349
4,emissions_ndc_month_g1_3,Green-recovery,World,Kyoto Gases (AR5-GWP100),Mt CO2-equiv/yr,2030,57624.744940


In [20]:
# Infill the desired components using interpolation between the nearest MESSAGE-GLOBIOM SSP2 scenarios
infiller_df = sr15_data.filter(model="MESSAGE-GLOBIOM*", scenario="SSP2*")

In [ ]:
cruncher = dc.LinearInterpolation(infiller_df)
desired_vars = []